# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГРГНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
from scipy import stats as st
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df_geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
df_geo_data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
df_geo_data_0.head(2)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.22117,105.280062
1,2acmU,1.334711,-0.340164,4.36508,73.037750


In [5]:
df_geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
df_geo_data_1.head(2)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261


In [7]:
df_geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
df_geo_data_2.head(2)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697


In [9]:
df_geo_data_0.duplicated().sum()

0

In [10]:
df_geo_data_0['id'].value_counts().head(11)

A5aEY    2
TtcGQ    2
Tdehs    2
HZww2    2
bsk9y    2
QcMuo    2
bxg6G    2
AGS9W    2
74z30    2
fiKDv    2
MpQQj    1
Name: id, dtype: int64

In [11]:
df_geo_data_0[df_geo_data_0['id'] == 'HZww2']

,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695


Одинаковые id  решил оставить т к их не так много

In [12]:
df_geo_data_1.duplicated().sum()

0

In [13]:
df_geo_data_2.duplicated().sum()

0

In [14]:
df_geo_data_0['product'].describe()

count    100000.000000
mean         92.500000
std          44.288691
min           0.000000
25%          56.497507
50%          91.849972
75%         128.564089
max         185.364347
Name: product, dtype: float64

In [15]:
df_geo_data_1['product'].describe()

count    100000.000000
mean         68.825000
std          45.944423
min           0.000000
25%          26.953261
50%          57.085625
75%         107.813044
max         137.945408
Name: product, dtype: float64

In [16]:
df_geo_data_2['product'].describe()

count    100000.000000
mean         95.000000
std          44.749921
min           0.000000
25%          59.450441
50%          94.925613
75%         130.595027
max         190.029838
Name: product, dtype: float64

Дубликатов в df не обнаружено. **NaN** тоже не обнаружено. так же посмотрели описательную статистику с помощью **describe()**.

## Обучение и проверка модели

In [17]:
df_geo_data_0 = df_geo_data_0.drop(['id'], axis = 1)
df_geo_data_0.head(2)

,f0,f1,f2,product
0,0.705745,-0.497823,1.22117,105.280062
1,1.334711,-0.340164,4.36508,73.037750


In [18]:
df_geo_data_1 = df_geo_data_1.drop(['id'], axis = 1)
df_geo_data_1.head(2)

,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261


In [19]:
df_geo_data_2 = df_geo_data_2.drop(['id'], axis = 1)
df_geo_data_2.head(2)

,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697


Удалил столбцы с типом данных **object** 

In [20]:
def we_keep_signs(df_geo_data):
    target = df_geo_data['product']  # Cохраняем целевой признак.
    features = df_geo_data.drop(['product'], axis=1)  # Cохраняем оставшиеся признаки, без целевого.
    
    return (target, features)

In [21]:
target_0, features_0 = we_keep_signs(df_geo_data_0)

In [22]:
target_1, features_1 = we_keep_signs(df_geo_data_1)

In [23]:
target_2, features_2 = we_keep_signs(df_geo_data_2)

In [24]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=69) # разбиваем df в соотношении 75 / 25

In [25]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=69) # разбиваем df в соотношении 75 / 25

In [26]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=69) # разбиваем df в соотношении 75 / 25

Масштабируем признаки :

In [27]:
def scaling_signs(features_train, features_valid):
    numeric = ['f0', 'f1', 'f2'] # Список признаков

    scaler = StandardScaler() # Создаём объект структуры 
    scaler.fit(features_train[numeric]) # Настроим его на обучающих данных

    features_train[numeric] = scaler.transform(features_train[numeric]) # Преобразуем обучающую выборку функцией transform()
    features_valid[numeric] = scaler.transform(features_valid[numeric]) # Преобразуем валидационную выборку функцией transform()
    pd.options.mode.chained_assignment = None
    
    return(features_train, features_valid)

In [28]:
features_train_0, features_valid_0 = scaling_signs(features_train_0, features_valid_0)

/tmp/ipykernel_83/2346915363.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric] = scaler.transform(features_train[numeric]) # Преобразуем обучающую выборку функцией transform()
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [29]:
features_train_1, features_valid_1 = scaling_signs(features_train_1, features_valid_1)

In [30]:
features_train_2, features_valid_2 = scaling_signs(features_train_2, features_valid_2)

In [31]:
def model_training(features_train, target_train, features_valid, target_valid, df):
    model = LinearRegression()
    model.fit(features_train, target_train) # обучаем модель
    predicted_valid = model.predict(features_valid) # предсказываем и сохраняем в переменную

    target_valid # правильные ответы для валидационной выборки 
    rmse = (mean_squared_error(target_valid, predicted_valid)) ** 0.5 # считаем rmse

    return predicted_valid

In [32]:
predicted_valid_0 = model_training(features_train_0, target_train_0, features_valid_0, target_valid_0, 'df_0')

In [33]:
display(f'Cредний запас предсказанного сырья для df_0: {predicted_valid_0.mean()}')
display(f'RMSE для df_0: {mean_squared_error(target_valid_0, predicted_valid_0) ** 0.5}') 

'Cредний запас предсказанного сырья для df_0: 92.55032321882531'

'RMSE для df_0: 37.71381030960315'

In [34]:
predicted_valid_1 = model_training(features_train_1, target_train_1, features_valid_1, target_valid_1, 'df_1')

In [35]:
display(f'Cредний запас предсказанного сырья для df_1: {predicted_valid_1.mean()}')
display(f'RMSE для df_1: {mean_squared_error(target_valid_1, predicted_valid_1) ** 0.5}') 

'Cредний запас предсказанного сырья для df_1: 69.00041524920387'

'RMSE для df_1: 0.8917870911060644'

In [36]:
predicted_valid_2 = model_training(features_train_2, target_train_2, features_valid_2, target_valid_2, 'df_2')

In [37]:
display(f'Cредний запас предсказанного сырья для df_2: {predicted_valid_2.mean()}')
display(f'RMSE для df_2: {mean_squared_error(target_valid_2, predicted_valid_2) ** 0.5}') 

'Cредний запас предсказанного сырья для df_2: 94.90813348489972'

'RMSE для df_2: 40.211342245620976'

In [38]:
def rmse_on_random_model(target_train, df):                                          # RMSE на рандомной модели
    predictions = pd.Series(target_train.mean(), index=target_train.index)
    rmse = (mean_squared_error(target_train, predictions)) ** 0.5

    return display(f'RMSE для {df}: {rmse}')

In [39]:
random_model_df_0 = rmse_on_random_model(target_train_0, 'df_0')

'RMSE для df_0: 44.28634389383188'

In [40]:
random_model_df_1 = rmse_on_random_model(target_train_1, 'df_1')

'RMSE для df_1: 45.950655877177255'

In [41]:
random_model_df_2 = rmse_on_random_model(target_train_2, 'df_2')

'RMSE для df_2: 44.72788288992564'

Очень большие **RMSE** в 0 и 2 регионах, в 1 регионе ошибка намного меньше чем в двух других. На рандомных и нормальных моделя, показатели по **RMSE** примерно одинаковы, но показатель **RMSE** в нормальной модели в 1 регионе, всего **1.53**, намного лучше всех остальных.

## Подготовка к расчёту прибыли

In [42]:
BUDGET = 10 ** 10
REVENUE_PER_UNIT_OF_PRODUCT = 450000
NUMBER_WELLS = 200

costs_per_1_well = BUDGET / NUMBER_WELLS # расходы на 1 скважину
budget = costs_per_1_well / REVENUE_PER_UNIT_OF_PRODUCT # бюджет на 1 скважину (тыс. баррелей)
budget

111.11111111111111

Вывод только один, что среднее сильно отличается от расчётного бюджета на 1 скважину, особенно во 2 регионе

In [43]:
predicted_valid_0.mean()

92.55032321882531

In [44]:
predicted_valid_1.mean()

69.00041524920387

In [45]:
predicted_valid_2.mean()

94.90813348489972

In [46]:
def profit_calculation(target_valid, predicted_valid):
    predicted_valid = pd.Series(predicted_valid).nlargest(NUMBER_WELLS)  # выбираем 200 лучших скважин
    result = target_valid.reset_index(drop=True)[predicted_valid.index]  # соотносим две таблицы по индексам
    return (result * REVENUE_PER_UNIT_OF_PRODUCT).sum() - BUDGET  # суммируем и рассчитываем прибыль для полученного объёма сырья

Функция, для подсчета прибыли 

## Расчёт прибыли и рисков 

In [47]:
def bootstrap_one(target, predict, name_col, name_col_risk_of_losses):
    state = np.random.RandomState(69)
    values = []
    bootstrap_samples = 1000
    
    for i in range(bootstrap_samples):
        predict_subsample = pd.Series(predict).sample(500, replace=False, random_state=state)
        values.append(profit_calculation(target, predict_subsample))
    
    values = pd.DataFrame(values, columns=[name_col])  # Делаем из списка DataFrame.
    df_negative_values = values[values[name_col] < 0]  # Выбираем значения меньше 0.
    df_positive_values = values.mean()  # Cчитаем среднее.
        
    lower = float(values.quantile(0.025))  
    upper = float(values.quantile(0.975))  
    
    risk_of_losses = df_negative_values.count() / values.count() # Считаем риск убытков.
    risk_of_losses = f'{float(risk_of_losses):.2%}'  # Переводим в проценты.
    
        
    return (lower, upper, df_positive_values, risk_of_losses)

In [48]:
lower_0, upper_0, df_positive_values_0, risk_of_losses_0 = bootstrap_one(target_valid_0, predicted_valid_0, 'df_0', 'df_0')

In [49]:
lower_1, upper_1, df_positive_values_1, risk_of_losses_1 = bootstrap_one(target_valid_1, predicted_valid_1, 'df_1', 'df_1')

In [50]:
lower_2, upper_2, df_positive_values_2, risk_of_losses_2 = bootstrap_one(target_valid_2, predicted_valid_2, 'df_2', 'df_2')

In [51]:
preparation_quantile = {'df_0': [lower_0, upper_0], 'df_1': [lower_1, upper_1], 'df_2': [lower_2, upper_2]}
pd.DataFrame(preparation_quantile, index=["0.025", "0.975"])

,df_0,df_1,df_2
0.025,-8.800263e+07,8.851749e+07,-1.075670e+08
0.975,8.865179e+08,8.701408e+08,9.096743e+08


95%-й доверительный интервал трёх регионов

In [52]:
preparation_risk = {'df_0': [risk_of_losses_0], 'df_1': [risk_of_losses_1], 'df_2': [risk_of_losses_2]}
pd.DataFrame(preparation_risk, index=["Риск убытков"])

,df_0,df_1,df_2
Риск убытков,6.20%,1.30%,5.30%


Риски убытков по трём регионам

In [53]:
pd.concat([df_positive_values_0, df_positive_values_1, df_positive_values_2]).to_frame(name='Средняя прибыль')

,Средняя прибыль
df_0,3.776999e+08
df_1,4.728763e+08
df_2,4.037946e+08


Средняя прибыль про трём регионам

## Вывод

В данном проекте работа велась над предсказанием прибыльности регионов, а так же скважин. В данной работе я сделал : 
- Разбил данные на обучающую и валидационную выборки в соотношении 75:25.
- Обучил модель и сделайте предсказания на валидационной выборке.
- Сохранил предсказания и правильные ответы на валидационной выборке.
- Напечатал на экране средний запас предсказанного сырья и RMSE модели.

Только RMSE для df_1: 0.8917870911060632 показал обнадеживающие результаты. Далее я :

- Рассчитал достаточный объём сырья для безубыточной разработки новой скважины. Сравнил полученный объём сырья со средним запасом в каждом регионе.
- Написал функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели.


- Посчитал риски и прибыль для каждого региона:

- Применив технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Нашел среднюю прибыль, 95%-й доверительный интервал и риск убытков. 

Из всего, можно сделать вывод, что район под названием df_1, можно выбрать для разработки скважин, т к Средняя прибыль на нем наибольшая - 4.803535e+08, Риск убытков самый маленький - 1.30%.